# Generative Adversarial Networks (GANs) - Basics

A **GAN (Generative Adversarial Network)** is made of two models:

1. **Generator** → Creates fake data (images, text, etc.)
2. **Discriminator** → Tries to distinguish between real and fake data

🔹 They compete in a **minimax game** until the generator produces realistic outputs.

📌 Applications:
- Image Generation (faces, art)
- Data Augmentation
- Style Transfer
- Super Resolution

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)

## Generator Model

In [ ]:
def build_generator(latent_dim):
    model = Sequential([
        Dense(128, input_dim=latent_dim),
        LeakyReLU(0.2),
        Dense(256),
        LeakyReLU(0.2),
        Dense(28*28, activation='tanh'),
        Reshape((28, 28, 1))
    ])
    return model

generator = build_generator(100)
generator.summary()

## Discriminator Model

In [ ]:
def build_discriminator():
    model = Sequential([
        Flatten(input_shape=(28,28,1)),
        Dense(256),
        LeakyReLU(0.2),
        Dense(128),
        LeakyReLU(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

discriminator = build_discriminator()
discriminator.summary()

## GAN Model (Generator + Discriminator)

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False  # freeze discriminator
    model = Sequential([generator, discriminator])
    model.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')
    return model

gan = build_gan(generator, discriminator)
gan.summary()

## Training Loop

In [ ]:
from tensorflow.keras.datasets import mnist

(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.0  # normalize to [-1,1]
X_train = np.expand_dims(X_train, axis=-1)

latent_dim = 100
epochs = 1000
batch_size = 64
half_batch = batch_size // 2

for epoch in range(epochs):
    # ----------------- Train Discriminator -----------------
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    real_imgs = X_train[idx]
    
    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_imgs = generator.predict(noise)
    
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # ----------------- Train Generator -----------------
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_y = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, valid_y)
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

## Generate New Images

In [ ]:
noise = np.random.normal(0, 1, (5, latent_dim))
gen_imgs = generator.predict(noise)

plt.figure(figsize=(10,2))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(gen_imgs[i, :, :, 0], cmap='gray')
    plt.axis('off')
plt.show()